In [1]:
!pip install matplotlib
!pip install seaborn
!pip install pandas
!pip install scipy
!pip install ctgan

In [2]:
# Let's read the CSV file into a pandas DataFrame using the given local file path.
import pandas as pd

# Path to the CSV file
file_path = 'combined_synthetic_data_with_patient_ids.csv'

# Read the CSV file into a DataFrame
combined_df = pd.read_csv(file_path)

# Show the first few rows of the DataFrame
combined_df.head()


,Datetime,Code,Activity_Type,Heart rate___beats/minute,Calories burned_kcal,Exercise duration_s,Sleep type duration_minutes,Sleep duration_minutes,Floors climbed___floors,Patient ID
0,2022=12-08 14:17,NaN,No Physical Activity,72.0,NaN,NaN,NaN,NaN,NaN,User0
1,2022=12-08 21:05,LA11836-6,Running,156.0,20.0,40851.0,NaN,NaN,NaN,User0
2,2022=12-08 22:01,LA11836-6,Running,170.0,30.0,44878.0,NaN,NaN,NaN,User0
3,2022=12-08 22:06,LA11836-6,Running,166.0,20.0,21497.0,NaN,NaN,NaN,User0
4,2022=12-08 22:16,NaN,No Physical Activity,78.0,NaN,NaN,NaN,NaN,NaN,User0


In [3]:
import pandas as pd
from ctgan import CTGAN

df = combined_df

# Drop non-numeric columns or any columns that won't be synthesized
df_cleaned = df.drop(columns=['Activity_Type','Patient ID','Code', 'Datetime','Sleep type duration_minutes','Sleep duration_minutes','Floors climbed___floors'])  # Adjust according to your needs

print(df_cleaned.head())
# Fill any missing values with the column mean or other appropriate strategies
df_cleaned.fillna(df_cleaned.mean(), inplace=True)

# Instantiate the CTGAN synthesizer
synthesizer = CTGAN()

# List of categorical columns (if any). Adjust based on your data
# categorical_columns = ['Code']  # You can add more categorical columns here

# Train the CTGAN model on the data
# synthesizer.fit(df_cleaned, discrete_columns=categorical_columns, epochs=300)
synthesizer.fit(df_cleaned,  epochs=5)

# Generate synthetic data (let's say you want to generate 1000 rows)
synthetic_data = synthesizer.sample(10)

# Convert the generated data to a DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=df_cleaned.columns)

# Save the synthetic data to a CSV file
synthetic_df.to_csv('synthetic_data_ctgan.csv', index=False)

print("Synthetic data saved to 'synthetic_data_ctgan.csv'.")


   Heart rate___beats/minute  Calories burned_kcal  Exercise duration_s
0                       72.0                   NaN                  NaN
1                      156.0                  20.0              40851.0
2                      170.0                  30.0              44878.0
3                      166.0                  20.0              21497.0
4                       78.0                   NaN                  NaN


C:\Users\fd\anaconda3\envs\torch\Lib\site-packages\torch\autograd\graph.py:768: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\cuda\CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Synthetic data saved to 'synthetic_data_ctgan.csv'.
